Defines and imports

In [ ]:
import json
import statistics
from datetime import datetime
from collections import defaultdict
import re
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PATH = ""
PATH = "../../dtn7-lab/shared/scenarios/correctness/others/results-correctness_others-1681488012/"

# Load path
# with open('.nbconfig', 'r') as file:
#     arguments = json.load(file)
#     PATH = arguments["path"]

print(f"Report for {PATH}")

with open(PATH + "start.txt") as file:
    simStart = int(file.readline().strip())
with open(PATH + "stop.txt") as file:
    simStop = int(file.readline().strip())


Resource Graphs

In [ ]:
def plotResourceGraphs():
    logfiles = []

    for root, _, files in os.walk(PATH):
        for file in files:
            if file in ["pidstat-base.csv.log", "pidstat-robot.csv.log"]:
                logfiles.append(root + "/" + file)

    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Base", "Robot"),
        specs=[[{"secondary_y": True}, {"secondary_y": True}]],
    )
    fig.update_xaxes(title_text="Time [s]")
    fig.update_yaxes(
        title_text="RSS [kB]",
        secondary_y=False,
        titlefont=dict(color="#ab63fa"),
        tickfont=dict(color="#ab63fa"),
    )
    fig.update_yaxes(
        title_text="CPU Usage [%]",
        secondary_y=True,
        titlefont=dict(color="#00cc96"),
        tickfont=dict(color="#00cc96"),
    )
    fig.update_layout(title_text="Resource Usage", showlegend=False)

    for file in logfiles:
        # Get node name
        nodeName = re.search(r"/pidstat-(\w*)", file).group(1)
        column = 1 if ("base" in nodeName) else 2

        # Get number of CPU Cores
        with open(file.replace(".csv", "")) as f:
            cpuCores = int(re.search(r"\((\d*) CPU\)", f.readline()).group(1))

        df = pd.read_csv(file, sep="\s+", usecols=[0, 7, 11, 12])
        # substract start time
        df = df.subtract([simStart, 0, 0, 0], axis="columns")
        # devide CPU usage by core count
        df = df.divide([1, cpuCores, 1, 1], axis="columns")

        # plot
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["RSS"],
                mode="lines",
                name="RSS",
                line=dict(color="#ab63fa", width=2),
            ),
            secondary_y=False,
            row=1,
            col=column,
        )
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["%CPU"],
                mode="lines",
                name="CPU",
                line=dict(color="#00cc96", width=2),
            ),
            secondary_y=True,
            row=1,
            col=column,
        )

        # Print mean values
        stats = f"Mean Values {nodeName}:\nCPU: \t{df.loc[:,'%CPU'].mean()} %\nRSS: \t{df.loc[:,'RSS'].mean()} kB\n"
        print(stats)

    fig.show()

plotResourceGraphs()


Network Graphs

In [ ]:
def plotNetworkGraphs():
    logfiles = {}

    # find log files
    for root, _, files in os.walk(PATH):
        for file in files:
            m = re.match(r"net-(.+).log", file)
            if m:
                logfiles[m.group(1)] = root + "/" + file

    # load data
    dfs = []
    for nodeName, file in logfiles.items():
        df = pd.read_csv(
            file,
            sep=";",
            usecols=[0, 1, 2, 3, 4],
            names="timestamp;iface_name;bytes_out/s;bytes_in/s;bytes_total/s".split(
                ";"
            ),
        )
        df["node"] = nodeName

        # filter
        df = df[df["iface_name"] == "eth0"]
        df = df[df["timestamp"] > simStart]

        # substract start time
        df["timestamp"] -= simStart
        df["timestamp"] = df["timestamp"].astype("int")

        dfs.append(df)

    df = pd.concat(dfs)
    df = df.sort_values(by=["timestamp", "node"])

    # init figure
    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        subplot_titles=("Traffic In", "Traffic Out", "Total Traffic"),
        vertical_spacing=0.1,
    )
    fig.update_xaxes(title_text="Time [s]")
    fig.update_yaxes(
        title_text="Traffic [B/s]",
    )
    fig.update_layout(title_text="Network Usage", height=900)

    # plot data
    colors = ["#ab63fa", "#00cc96", "#ffa15a"]
    colorCnt = 0
    for nodeName in df.node.unique():
        nodeDf = df[df.node == nodeName]

        fig.add_trace(
            go.Scatter(
                x=nodeDf["timestamp"],
                y=nodeDf["bytes_in/s"],
                mode="lines",
                name=nodeName,
                line=dict(color=colors[colorCnt]),
                legendgroup=colorCnt,
                showlegend=True,
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=nodeDf["timestamp"],
                y=nodeDf["bytes_out/s"],
                mode="lines",
                name=nodeName,
                line=dict(color=colors[colorCnt]),
                legendgroup=colorCnt,
                showlegend=False,
            ),
            row=2,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=nodeDf["timestamp"],
                y=nodeDf["bytes_total/s"],
                mode="lines",
                name=nodeName,
                line=dict(color=colors[colorCnt]),
                legendgroup=colorCnt,
                showlegend=False,
            ),
            row=3,
            col=1,
        )

        colorCnt += 1

    # plot means
    meanIn = df["bytes_in/s"].describe().T["mean"]
    meanOut = df["bytes_out/s"].describe().T["mean"]
    meanTotal = df["bytes_total/s"].describe().T["mean"]

    fig.add_hline(
        meanIn,
        line=dict(color="red", dash="dash", width=1),
        row=1,
        col=1,
    )
    fig.add_hline(
        meanOut,
        line=dict(color="red", dash="dash", width=1),
        row=2,
        col=1,
    )
    fig.add_hline(
        meanTotal,
        line=dict(color="red", dash="dash", width=1),
        row=3,
        col=1,
    )

    fig.show()

    print("(Combined Nodes) Mean Network usage:")
    print(
        f"In: \t{meanIn:.2f} B/s\nOut: \t{meanOut:.2f} B/s\nTotal: \t{meanTotal:.2f} B/s"
    )

    print("\nTotal Traffic:")
    for nodeName in df.node.unique():
        nodeDf = df[df.node == nodeName]
        print(
            f"{nodeName} \tIn: {str(sum(nodeDf['bytes_in/s'])/1000) + ' kB':<15} Out: {sum(nodeDf['bytes_out/s'])/1000} kB"
        )


plotNetworkGraphs()


Parsing dtnd stats

In [ ]:
dtndStats = {"base": {}, "robot": {}, "combined": {}}


def loadDtndStats():
    global dtnStats

    # init
    for node in dtndStats:
        dtndStats[node]["sentBundles"] = {}
        dtndStats[node]["receivedBundles"] = {}

    for root, _, files in os.walk(PATH):
        for file in files:
            if file != "dtnd.log":
                continue
            nodeName = re.search(r"\/(\w*)$", root).group(1)
            with open(root + "/" + file) as f:
                content = f.read()

            if nodeName in ["base", "robot"]:

                dtndStats[nodeName]["created"] = len(re.findall(r"Transmission of bundle requested", content))
                dtndStats[nodeName]["transferred"] = len(re.findall(r"Sending bundle succeeded", content))
                dtndStats[nodeName]["relayed"] = len(re.findall(r"Received new bundle", content))
                dtndStats[nodeName]["aborted"] = len(re.findall(r"Sending bundle .+ failed", content))
                dtndStats[nodeName]["dropped"] = len(re.findall(r"Dropping bundle", content))
                dtndStats[nodeName]["removed"] = len(re.findall(r"Removing bundle", content)) - dtndStats[nodeName]["dropped"]
                dtndStats[nodeName]["refused"] = len(re.findall(r"refusing bundle", content))
                dtndStats[nodeName]["delivered"] = len(re.findall(r"Received bundle for local delivery", content))

                tmp = re.findall(r"(\d{4}-\d{2}-\d{2}T.+Z).*Received bundle for local delivery.+\/\/(.+)", content)
                for time, bid in tmp:
                    dtndStats[nodeName]["receivedBundles"][bid] = datetime.fromisoformat(time).timestamp()
                tmp = re.findall(r"(\d{4}-\d{2}-\d{2}T.+Z).*Transmission of bundle requested.+\/\/(.+)", content)
                for time, bid in tmp:
                    dtndStats[nodeName]["sentBundles"][bid] = datetime.fromisoformat(time).timestamp()

            try:
                dtndStats["combined"]["transferred"] = dtndStats["combined"]["transferred"] + len(re.findall(r"Sending bundle succeeded", content))
            except (KeyError):
                dtndStats["combined"]["transferred"] = len(re.findall(r"Sending bundle succeeded", content))

loadDtndStats()


DTN Graphs

In [ ]:
def dtnLatency():
    global dtndStats

    sentBundles = {
        **dtndStats["base"]["sentBundles"],
        **dtndStats["robot"]["sentBundles"],
    }
    receivedBundles = {
        **dtndStats["base"]["receivedBundles"],
        **dtndStats["robot"]["receivedBundles"],
    }

    # Calculate latency
    latencyMap = defaultdict(list)

    # To plot over time, latency values of bundles received in one second are combined
    for bid, time in receivedBundles.items():
        sTime = sentBundles[bid]
        latency = time - sTime
        latencyMap[int(time) + 1].append(latency)
    latencyOverTime = {}
    for time, val in latencyMap.items():
        latencyOverTime[time - simStart] = sum(val) / len(val)

    fig = go.Figure()
    fig.update_xaxes(title_text="Time [s]", range=[0, simStop - simStart])
    fig.update_yaxes(
        title_text="DTN Latency [s]",
        titlefont=dict(color="#ab63fa"),
        tickfont=dict(color="#ab63fa"),
    )
    fig.update_layout(title_text="Average DTN Latency over all Topics")
    fig.add_trace(
        go.Scatter(
            x=[*latencyOverTime.keys()],
            y=[*latencyOverTime.values()],
            mode="lines+markers",
            line=dict(color="#ab63fa", width=2),
        ),
    )
    fig.show()

    # calc mean latency
    total = 0
    count = 0
    for val in latencyMap.values():
        total += sum(val)
        count += len(val)

    print(f"Mean Latency: {total/count}s")

def sumUntilKey(d, key):
    total = 0
    for i in range(0, key + 1):
        total += d[i]
    return total

def dtnBundlesInFlight():
    sentBundles = {
        **dtndStats["base"]["sentBundles"],
        **dtndStats["robot"]["sentBundles"],
    }
    receivedBundles = {
        **dtndStats["base"]["receivedBundles"],
        **dtndStats["robot"]["receivedBundles"],
    }

    bundlesMap = defaultdict(lambda: 0)
    for time in sentBundles.values():
        bundlesMap[int(time) + 1 - simStart] += 1
    for time in receivedBundles.values():
        bundlesMap[int(time) + 1 - simStart] -= 1

    result = []
    for time in range(0, simStop - simStart + 1):
        result.append(sumUntilKey(bundlesMap, time))

    # plot graph
    fig = go.Figure()
    fig.update_xaxes(title_text="Time [s]", range=[0, simStop - simStart])
    fig.update_layout(title_text="Number of bundles in flight")
    fig.add_trace(
        go.Scatter(
            y=result,
            mode="lines+markers",
            line=dict(color="#ab63fa", width=2),
        ),
    )
    fig.show()


def dtnBundleCount():
    print(f"\nTransfered bundles over all nodes: {dtndStats['combined']['transferred']}")


dtnLatency()
dtnBundlesInFlight()
dtnBundleCount()


ROS2DTN Proxy

End-To-End Latency

In [ ]:
def parseE2eLatency():
    sentMsgs = defaultdict(lambda: defaultdict(lambda: 0))
    receivedMsgs = defaultdict(lambda: defaultdict(lambda: 0))

    for root, _, files in os.walk(PATH):
        if not any(name in root for name in ["robot", "base"]):
            continue

        for file in files:
            if not re.search(r"\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}\.log", file):
                continue
            with open(root + "/" + file) as f:
                content = f.readlines()

                for line in content:
                    if not ";ROS;" in line:
                        continue
                    timestamp, _, _, _, topic, _, _, msgHash = line.split(";")
                    if ";ROS;TOPIC;PUB;" in line:
                        receivedMsgs[topic][msgHash.strip()] = datetime.fromisoformat(timestamp).timestamp()
                    elif "ROS;TOPIC;SUB;" in line:
                        sentMsgs[topic][msgHash.strip()] = datetime.fromisoformat(timestamp).timestamp()

    
    return (sentMsgs, receivedMsgs)

def e2eLatency(data):

    sentMsgs, receivedMsgs = data

    for topic in sentMsgs.keys():
        # Calculate latency
        latencyMap = defaultdict(list)

        # To plot over time, latency values of bundles received in one second are combined
        for bid, time in receivedMsgs[topic].items():
            sTime = sentMsgs[topic][bid]
            latency = time - sTime
            latencyMap[int(time) + 1].append(latency)
        latencyOverTime = {}
        for time, val in latencyMap.items():
            latencyOverTime[time - simStart] = sum(val) / len(val)

        fig = go.Figure()
        fig.update_xaxes(title_text="Time [s]", range=[0, simStop - simStart])
        fig.update_yaxes(
            title_text="E2E Latency [s]",
            titlefont=dict(color="#ab63fa"),
            tickfont=dict(color="#ab63fa"),
        )
        fig.update_layout(title_text=f"Average End-To-End Latency for topic {topic}")
        fig.add_trace(
            go.Scatter(
                x=[*latencyOverTime.keys()],
                y=[*latencyOverTime.values()],
                mode="lines+markers",
                line=dict(color="#ab63fa", width=2),
            ),
        )
        fig.show()

        # calc mean & stddev
        latencies = []
        for val in latencyMap.values():
            latencies.extend(val)

        print(f"Mean Latency: {statistics.mean(latencies)}s")
        print(f"StDev: {statistics.stdev(latencies)}s")

e2eLatency(parseE2eLatency())

Pipeline Latency

In [ ]:
def latencyInputPipeline():
    result = {}
    count = defaultdict(lambda: 0)
    for root, _, files in os.walk(PATH):
        if not any(name in root for name in ["robot"]):
            continue

        for file in files:
            if not re.search(r"\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}\.log", file):
                continue
            with open(root + "/" + file) as f:
                content = f.readlines()
            
            for index, line in enumerate(content):
                if "DTN;TOPIC" in line:
                    end, _, _, _, endTopic, _, _ = line.split(";")
                    end = end.split("+")[0]
                    start, _, _, _, startTopic, _, _, _ = content[index - 1].split(";")
                    start = start.split("+")[0]
                    if endTopic == startTopic:
                        result.setdefault(endTopic, [])
                        diff = datetime.fromisoformat(end).timestamp() - datetime.fromisoformat(start).timestamp()
                        result[endTopic].append(diff)
                    else:
                        raise ValueError("Topics should be the same!")
                elif "ROS;TOPIC" in line:
                    topic = line.split(";")[4]
                    count[topic] += 1
    
    print("Input Pipline Latency")
    for topic, values in result.items():
        print(f"{topic}:\tSamples: {count[topic]}\tMean: {statistics.mean(values)}\t StDev: {statistics.stdev(values)}")

def latencyOutputPipeline():
    result = {}
    count = defaultdict(lambda: 0)
    for root, _, files in os.walk(PATH):
        if not any(name in root for name in ["base"]):
            continue

        for file in files:
            if not re.search(r"\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}\.log", file):
                continue
            with open(root + "/" + file) as f:
                content = f.readlines()
            
            for index, line in enumerate(content):
                if "DTN;TOPIC" in line:
                    start, _, _, _, startTopic, _, _ = line.split(";")
                    start = start.split("+")[0]
                    end, _, _, _, endTopic, _, _, _ = content[index + 1].split(";")
                    end = end.split("+")[0]
                    if endTopic == startTopic:
                        result.setdefault(endTopic, [])
                        diff = datetime.fromisoformat(end).timestamp() - datetime.fromisoformat(start).timestamp()
                        result[endTopic].append(diff)
                        count[endTopic] += 1
                    else:
                        raise ValueError("Topics should be the same!")
    
    print("\nOutput Pipline Latency")
    for topic, values in result.items():
        print(f"{topic}:\tSamples: {count[topic]}\tMean: {statistics.mean(values)}\t StDev: {statistics.stdev(values)}")

latencyInputPipeline()
latencyOutputPipeline()

In [ ]:
def parseProxyStats():
    # TODO: replace dtn values on sender with ros values on receiver
    topics = set()
    sizeDtn = {}
    sizeRos = {}
    cntDtn = {}
    cntRos = {}

    for root, _, files in os.walk(PATH):
        if not any(name in root for name in ["base", "robot"]):
            continue

        for file in files:
            if not re.search(r"\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}\.log", file):
                continue
            with open(root + "/" + file) as f:
                content = f.readlines()

            for line in content:
                # only use subscriptions to prevent duplication
                if not ";TOPIC;SUB;" in line:
                    continue
                timestamp, tech, _, _, topic, type, size, *_ = line.split(";")
                # timestamp = datetime.fromisoformat(timestamp).timestamp()
                topics.add(topic)
                sizeDtn.setdefault(topic, [])
                sizeRos.setdefault(topic, [])
                cntDtn.setdefault(topic, 0)
                cntRos.setdefault(topic, 0)

                if "DTN" == tech:
                    sizeDtn[topic].append(int(size))
                    cntDtn[topic] += 1
                elif "ROS" == tech:
                    sizeRos[topic].append(int(size))
                    cntRos[topic] += 1

    for topic, values in sizeDtn.items():
        total = sum(values)
        sizeDtn[topic] = total / len(values)

    for topic, values in sizeRos.items():
        total = sum(values)
        sizeRos[topic] = total / len(values)
    
    sizeDtn = sorted(sizeDtn.items(), key=lambda x: x[0])
    sizeRos = sorted(sizeRos.items(), key=lambda x: x[0])
    cntDtn = sorted(cntDtn.items(), key=lambda x: x[0])
    cntRos = sorted(cntRos.items(), key=lambda x: x[0])

    topics = [item[0] for item in sizeDtn]
    sizeDtn = [item[1] for item in sizeDtn]
    sizeRos = [item[1] for item in sizeRos]
    cntDtn = [item[1] for item in cntDtn]
    cntRos = [item[1] for item in cntRos]

    return (topics, cntRos, cntDtn, sizeRos, sizeDtn)


def dtnProxyStats(parsedData):
    topics, cntRos, cntDtn, sizeRos, sizeDtn = parsedData

    fig = make_subplots(
        rows=1, cols=2, subplot_titles=("Message Count", "Message Size")
    )

    fig.update_layout(barmode="group", title_text="Bidirectional Message Statistics")
    fig.update_yaxes(title_text="Message Size [B]", row=1, col=2)
    fig.update_yaxes(title_text="Message Count", row=1, col=1)

    fig.add_trace(
        go.Bar(
            name="ROS",
            x=topics,
            y=cntRos,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#ab63fa"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            name="DTN",
            x=topics,
            y=cntDtn,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#00cc96"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            name="ROS",
            x=topics,
            y=sizeRos,
            texttemplate="%{y:.1f}",
            textposition="outside",
            marker=dict(color="#ab63fa"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Bar(
            name="DTN",
            x=topics,
            y=sizeDtn,
            texttemplate="%{y:.1f}",
            textposition="outside",
            marker=dict(color="#00cc96"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    fig.show()

    # print stats per topic
    # bandwidth
    print("Transferred Bandwith (DTN payload ONLY)")
    for topic, cnt, size in zip(topics, cntDtn, sizeDtn):
        print(f"{topic + ':':<20} \t{(cnt * size) / 1000} kB")

    # count
    print("\nMessage Count reduction (ROS -> DTN)")
    for topic, ros, dtn in zip(topics, cntRos, cntDtn):
        print(f"{topic + ':':<20} \t{ros - dtn} = {((ros-dtn)/ros*100):.2f}%")

    # size
    print("\nMessage Size reduction (ROS -> DTN)")
    for topic, ros, dtn in zip(topics, sizeRos, sizeDtn):
        print(f"{topic + ':':<25}{f'{(ros - dtn):.2f}':<10} Bytes = {((ros-dtn)/ros*100):05.2f}%")

parsedData = parseProxyStats()
dtnProxyStats(parsedData)
